# Get Bee Hive Data

In [1]:
import glob
import numpy as np
import pandas as pd

In [2]:
hives = glob.glob("data/*.txt")
df = pd.DataFrame(columns=["date", "weight", "hive_temp", "hive_hum", "out_temp", 
                            "out_hum", "1", "2", "3", "4", "5", "6", "7", "8", 
                            "9", "10", "11"])
for hive in hives:
    temp = pd.read_csv(hive, 
                     names=["date", "weight", "hive_temp", "hive_hum", "out_temp", 
                            "out_hum", "1", "2", "3", "4", "5", "6", "7", "8", 
                            "9", "10", "11"])
    df = df.append(temp)

In [3]:
df = df.drop(["date", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11"], axis = 1)

In [4]:
df = df.dropna()

# Normalize Data

In [5]:
from sklearn.preprocessing import StandardScaler

In [6]:
x_scaled = df.values.astype("float")
scaler = StandardScaler()
scaler.fit(x_scaled)
x_scaled = scaler.transform(x_scaled)
df_scaled = pd.DataFrame(x_scaled)

# Reshape Model for Model

In [7]:
def make_windows(data, size):
    X = []
    Y = []
    i = 0
    while(i+size) <= len(data)-1:
        X.append(data[i:i+size])
        Y.append(data[i + size])
        i += 1
        
    return X,Y

In [8]:
train = x_scaled[0 : int(len(x_scaled)*.8)]
test  = x_scaled[int(len(x_scaled)*.8) : len(x_scaled)]

In [9]:
X_train, Y_train = make_windows(train, 32)
X_test, Y_test = make_windows(train, 32)

In [10]:
X_train = np.array(X_train).reshape(len(X_train), len(X_train[0]), 5)
X_test = np.array(X_test).reshape(len(X_test), len(X_test[0]), 5)

In [11]:
Y_train = np.array(Y_train).reshape(-1,5)
Y_test = np.array(Y_test).reshape(-1,5)

In [12]:
print(Y_train[0])

[-1.16304012 -0.52874706  0.38280518  0.50429881  0.24341347]


# Build Model (Supervised RNN)

In [13]:
import tensorflow
from keras.layers import LSTM, Dense, Dropout, TimeDistributed
from keras.models import Sequential

C:\Users\Adrian\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [14]:
model = Sequential()
model.add(LSTM(16, input_shape=(32,5), return_sequences=True))
model.add(LSTM(16, return_sequences=True))
model.add(LSTM(8, return_sequences=False))
model.add(Dense(5))
model.compile(optimizer='adam', loss='mse')

In [15]:
model.fit(X_train, Y_train, epochs = 128, verbose = 1)

Epoch 1/128
9057/9057 [==============================] - 9s 976us/step - loss: 0.6247
Epoch 2/128
9057/9057 [==============================] - 7s 778us/step - loss: 0.4072
Epoch 3/128
9057/9057 [==============================] - 7s 764us/step - loss: 0.3357
Epoch 4/128
9057/9057 [==============================] - 7s 774us/step - loss: 0.2978
Epoch 5/128
9057/9057 [==============================] - 7s 770us/step - loss: 0.2734
Epoch 6/128
9057/9057 [==============================] - 7s 764us/step - loss: 0.2539
Epoch 7/128
9057/9057 [==============================] - 7s 774us/step - loss: 0.2396
Epoch 8/128
9057/9057 [==============================] - 7s 771us/step - loss: 0.2279
Epoch 9/128
9057/9057 [==============================] - 7s 771us/step - loss: 0.2168
Epoch 10/128
9057/9057 [==============================] - 7s 787us/step - loss: 0.2081
Epoch 11/128
9057/9057 [==============================] - 7s 774us/step - loss: 0.2025
Epoch 12/128
9057/9057 [============================

9057/9057 [==============================] - 7s 784us/step - loss: 0.1270
Epoch 96/128
9057/9057 [==============================] - 7s 770us/step - loss: 0.1278
Epoch 97/128
9057/9057 [==============================] - 7s 793us/step - loss: 0.1263
Epoch 98/128
9057/9057 [==============================] - 7s 779us/step - loss: 0.1262
Epoch 99/128
9057/9057 [==============================] - 7s 768us/step - loss: 0.1255
Epoch 100/128
9057/9057 [==============================] - 7s 775us/step - loss: 0.1251
Epoch 101/128
9057/9057 [==============================] - 7s 799us/step - loss: 0.1247
Epoch 102/128
9057/9057 [==============================] - 7s 771us/step - loss: 0.1262
Epoch 103/128
9057/9057 [==============================] - 7s 769us/step - loss: 0.1265
Epoch 104/128
9057/9057 [==============================] - 7s 785us/step - loss: 0.1253
Epoch 105/128
9057/9057 [==============================] - 7s 770us/step - loss: 0.1244
Epoch 106/128
9057/9057 [=========================

In [16]:
model.evaluate(X_test, Y_test, verbose = 1)

9057/9057 [==============================] - 2s 170us/step


0.1160883827697853

In [17]:
print(Y_test[0])

[-1.16304012 -0.52874706  0.38280518  0.50429881  0.24341347]


In [19]:
print(model.predict(X_test[0].reshape(1, 32, 5)))

[[-1.0777938  -0.46479934  0.4555275   0.52240217  0.33280346]]
